<a href="https://colab.research.google.com/github/hanjj753/Market_Prediction/blob/main/Market%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Market Prediction 데이터톤용 구조 코드

## 1. 파일 다운로드
- 아래 코드들을 실행해 필요한 파일들 다운로드 및 패키지 설치

In [ ]:
!git clone https://github.com/hanjj753/Market_Prediction.git

## 2. 참고용 예시 코드

## 3. 실제 사용할 코드